# Ebay Kleinenzeigen Popular Brand Car Sales Analysis 

The project aims to find most popular car brands and their average price and average kilometers, meaning when users decide to sell their popular brand cars at what kilometer. A chunk of [Ebay Kleinenzeigen Car Sales Analysis](https://www.kaggle.com/orgesleka/used-cars-database/data) on Kaggle is used as data source.

In [1]:
import pandas as pd
import numpy as np

autos = pd.read_csv('Data\\autos.csv',encoding='Latin-1')

autos.info()
autos.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


As can be seen above, the data set consist of 20 columns and in total 50,000 random samples from the data set on the Kaggle.  Below are column descriptions of the data set.


* dateCrawled : When the ad was first crawled.
* name : name of the car
* seller : Seller of the car, private or handler.
* offerType : type of listing
* price : price of the car
* abtest : if the listing is included in an A/B test
* yearOfRegistration : registration year of car
* gearbox : transmission type
* powerPS : transmission type automatic/manuel
* model : car model
* kilometer : how many kilometers the car has driven
* monthOfRegistration : registration month of car
* fuelType : Fuel type of car, e.g Diesel
* brand : car brand
* notRepairedDamage : if car has not repaired damage
* dateCreated : ad creation date
* nrOfPictures : number of pictures in the ad
* postalCode : post code of car location
* lastSeenOnline : when the crawler saw this ad last online

# Data Cleaning

### Column Name Corrections
As can be seen above, some of the column names does not follow snakecase, let's find out and replace them into snakecase.

In [2]:
autos.columns

autos.rename(columns = {
                        "yearOfRegistration":"registration_year",
                        "monthOfRegistration":"registration_month",
                        "notRepairedDamage":"unrepaired_damage",
                        "monthOfRegistration":"registration_month",
                        "notRepairedDamage":"unrepaired_damage",
                        "dateCrawled":"date_crawled",
                        "offerType":"ad_type",
                        "vehicleType":"car_type",
                        "fuelType":"fuel_type",
                        "nrOfPictures":"im_count",
                        "postalCode":"post_code",
                        "lastSeen":"last_seen",
                        "powerPS":"power_ps",
                        "dateCreated":"ad_created"},
                     inplace=True)

autos.head()


,date_crawled,name,seller,ad_type,price,abtest,car_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,im_count,post_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


### Basic Data Exploration

At this step, data set will be explored roughly with the help of describe() function. As can be seen **price** and **odometer** column contains numeric values but they stored as strings becuase of the "$" sign and "km"

In [3]:
autos.describe(include='all')

,date_crawled,name,seller,ad_type,price,abtest,car_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,im_count,post_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-16 21:50:53,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Let's convert price and odometer columns into integer values, also rename them.

In [4]:
autos["price"] = autos["price"].str.replace("$","").str.replace(",","").astype(int)

autos["odometer"] = autos["odometer"].str.replace("km","").str.replace(",","").astype(int)

autos.rename(columns={"price":"price_usd","odometer":"odometer_km"},inplace=True)

### Removing Outliers
Removing outliers from the data has a significant effect on our analysis as they are removed from the data to be used during analysis. This is why, I need to minimize data loss.


Let's explore price_usd column to calculate outliers, meaning any price value that significantly differs from others. To do that, I am going to sort the values and check where outlier occurs. As can be seen below, price values suddenly jumped from 350000 to 999990.

In [5]:
autos["price_usd"].value_counts().sort_index(ascending=True).tail(20)

197000      1
198000      1
220000      1
250000      1
259000      1
265000      1
295000      1
299000      1
345000      1
350000      1
999990      1
999999      2
1234566     1
1300000     1
3890000     1
10000000    1
11111111    2
12345678    3
27322222    1
99999999    1
Name: price_usd, dtype: int64

Time to remove the outliers, any price below 1 and above 350000 should be removed.

In [6]:
autos = autos[autos["price_usd"].between(1,350001)]
autos["price_usd"].value_counts().sort_index(ascending=True).tail(20)

151990    1
155000    1
163500    1
163991    1
169000    1
169999    1
175000    1
180000    1
190000    1
194000    1
197000    1
198000    1
220000    1
250000    1
259000    1
265000    1
295000    1
299000    1
345000    1
350000    1
Name: price_usd, dtype: int64

When we explore odometer_km column, it's clearly visible that there is no outlier.

In [7]:
autos["odometer_km"].value_counts()

150000    31414
125000     5057
100000     2115
90000      1734
80000      1415
70000      1217
60000      1155
50000      1012
5000        836
40000       815
30000       780
20000       762
10000       253
Name: odometer_km, dtype: int64

In [8]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48565 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   date_crawled        48565 non-null  object
 1   name                48565 non-null  object
 2   seller              48565 non-null  object
 3   ad_type             48565 non-null  object
 4   price_usd           48565 non-null  int32 
 5   abtest              48565 non-null  object
 6   car_type            43979 non-null  object
 7   registration_year   48565 non-null  int64 
 8   gearbox             46222 non-null  object
 9   power_ps            48565 non-null  int64 
 10  model               46107 non-null  object
 11  odometer_km         48565 non-null  int32 
 12  registration_month  48565 non-null  int64 
 13  fuel_type           44535 non-null  object
 14  brand               48565 non-null  object
 15  unrepaired_damage   39464 non-null  object
 16  ad_created          48

In [9]:
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

In [10]:
autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038855
2016-04-04    0.036858
2016-04-05    0.011819
2016-04-06    0.003253
2016-04-07    0.001256
Name: ad_created, Length: 76, dtype: float64

In [11]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

In [12]:
autos["registration_year"].value_counts().sort_index(ascending=True)

1000    1
1001    1
1111    1
1800    2
1910    5
       ..
5911    1
6200    1
8888    1
9000    1
9999    3
Name: registration_year, Length: 95, dtype: int64

As can be seen above,there are some values that does not look like realistic. Determining earliest valid year is hard but I am going to take 1920 as minimum and 2016 as maximum (As data crawled in 2016)



In [24]:
autos = autos[autos["registration_year"].between(1920,2016)]
autos["registration_year"].value_counts(normalize=True, dropna=False).sort_index()

1927    0.000021
1929    0.000021
1931    0.000021
1934    0.000043
1937    0.000086
          ...   
2012    0.028066
2013    0.017204
2014    0.014204
2015    0.008398
2016    0.026138
Name: registration_year, Length: 77, dtype: float64

### What is the most common registration year?

Below, I am going to find the most common registration year, meaning what is the most frequent registration year in the data set.

***2000 model cars are the most which are on sale.***

In [26]:
autos["registration_year"].mode()

0    2000
dtype: int64

## Data Aggregation

The data crawled from a very popular e-commerce web site and it's expected to have multiple car brands with their different models. To aggregate our analysis, I am going to answer the following questions.

* **How to find out what car brands are the most popular in the site?**
   
   In order to find what car brands are the most popular, I am going to select brands where they occure more than 5% in the data set.
* **What are the most popular car brands?**
   
    e-Bay Kleinenzeigen operates in Germany and the result below is not surprising. Volkswagen, BMW, Opel, Mercedes-Benz, Audi and Ford are the most frequent car brands in the data set.
* **What are the average prices per popular car brand?**

    Audi, BMW and Mercedes-Benz are more expensive if compare to Opel, Ford and Volkswagen. The cheapest brand (in terms of mean price) is Opel.




In [38]:
car_brands= autos["brand"].value_counts(normalize=True)
popular_brands = car_brands[car_brands>0.05].index

popular_mean_prices= {}
for brand in popular_brands:
    selected_brand = autos[autos["brand"]==brand]
    price_mean= selected_brand["price_usd"].mean()
    popular_mean_prices[brand] = int(price_mean)
popular_mean_prices

{'volkswagen': 5402,
 'bmw': 8332,
 'opel': 2976,
 'mercedes_benz': 8628,
 'audi': 9336,
 'ford': 3749}

Let's take one more step further in our analysis and chceck at what kilometer people decide to sell their cars.

In [ ]:
popular_mean_km = {}

for brand in popular_brands:
    selected_brand= autos[autos["brand"]==brand]
    km_mean = selected_brand["odometer_km"].mean()
    popular_mean_km[brand] = int(km_mean)

# Conclusion

In this project, my aim was to answer the following question

**At what kilometer popular car brand owners decide to sell their cars?**

As can be seen dataframe below, Audi owners are selling their car aroung 130.000 km for the most expensive price, than Mercedes-Benz, BMW, Volkswagen, Opel and Ford.

We can also conclude that people are trying to sell their cars around when they drive 125.000 - 130.000 km.

In [53]:
price_series = pd.Series(popular_mean_prices)
km_series = pd.Series(popular_mean_km)

autos_popular_brand = pd.DataFrame({'mean_price_usd':price_series,'mean_km':km_series})
autos_popular_brand.sort_values(by='mean_km')

,mean_price_usd,mean_km
ford,3749,124266
volkswagen,5402,128707
audi,9336,129157
opel,2976,129311
mercedes_benz,8628,130788
bmw,8332,132572
